In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
data_dir = "~/tmp/data"
cora = Planetoid(root=data_dir, name="Cora")

print(f"Number of classes: {cora.num_classes}")
print(f"Number of features: {cora.num_features}")
print(f"Number of nodes: {cora.data.num_nodes}")
print(f"Number of edges: {cora.data.num_edges}")
print(f"Training nodes: {cora.data.train_mask.sum()}")
print(f"Val nodes: {cora.data.val_mask.sum()}")
print(f"Test nodes: {cora.data.test_mask.sum()}")

In [ ]:
print(cora.data.y[0])
print(cora.data.x.shape)
print(cora.data.train_mask)

In [ ]:
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(cora.num_features, 128)
        self.conv2 = GCNConv(128, 32)
        self.conv3 = GCNConv(32, cora.num_classes)
                
    def forward(self, h, edge_index):
        h = self.conv1(h, edge_index)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.conv2(h, edge_index)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.conv3(h, edge_index)
        
        return h

In [ ]:
def train(model, data, epochs, epoch_log, criterion, optimizer):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            model.eval()
            val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                    
        if epoch % epoch_log == 0:
            print(f"epoch {epoch} - train loss: {loss} - val loss: {val_loss}")
            
def test(model, data):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    results = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = results / data.test_mask.sum()
    print(acc)

In [ ]:
epochs = 1000
epoch_log = 10
weight_decay = 5e-3
lr = 0.0001
criterion = nn.CrossEntropyLoss()

model = GCN().to(device)
data = cora.data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
print(f"Using {device} to train.")
train(model, data, epochs, epoch_log, criterion, optimizer)

In [ ]:
test(model, data)